In [ ]:
#import sys dependancy to change path for config file
import sys
sys.path.insert(0, '..')

In [ ]:
# import dependencies
import googlemaps
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from config import sqlpassword, password, gmaps_password
from datetime import datetime, timedelta, date
import openpyxl
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
#define the distance for the radius calculation
distance = 30
#define how far back historical data will be pulled
timespan = 365

In [ ]:
#connect to the google maps api with an API key
gmaps = googlemaps.Client(key=f"{gmaps_password}")

In [ ]:
#define a start time to see how long the script takes to run
start = datetime.now()

In [ ]:
#define an end date
end_date = date.today()

In [ ]:
#Define a dictionary of key value pairs that accepts a month and returns the first month of the quarter
quarter_start = {12:{'Start':10, 'Quarter':'Q4'},
                 11:{'Start':10, 'Quarter':'Q4'},
                 10:{'Start':10, 'Quarter':'Q4'},
                 9:{'Start':7, 'Quarter':'Q3'},
                 8:{'Start':7, 'Quarter':'Q3'},
                 7:{'Start':7, 'Quarter':'Q3'},
                 6:{'Start':4, 'Quarter':'Q2'},
                 5:{'Start':4, 'Quarter':'Q2'},
                 4:{'Start':4, 'Quarter':'Q2'},
                 3:{'Start':1, 'Quarter':'Q1'},
                 2:{'Start':1, 'Quarter':'Q1'},
                 1:{'Start':1, 'Quarter':'Q1'}}

In [ ]:
#define a start date that's the beginning of the past 4 full quarters.  
#Will include 4 full quarters and the current partial quarter.
start_date = end_date.replace(year = end_date.year - 1, month = quarter_start[end_date.month]['Start'], day = 1)

In [ ]:
# connect to the server through engine
engine = create_engine(f'{sqlpassword}')
conn = engine.connect()

In [ ]:
# read in the historical data as a dataframe
historical_data = pd.read_sql(f"SELECT id, origin_city, origin_state, pickup_zip_code, dest_city, \
                         dest_state, consignee_zip_code, pickup_date, carrier_line_haul, miles,\
                             CASE equipment WHEN 'V' THEN 'V' \
                             WHEN 'VF' THEN 'V' \
                             WHEN 'VR' THEN 'V' \
                             WHEN 'F' THEN 'F' \
                             WHEN 'FSD' THEN 'F' \
                             WHEN 'FT' THEN 'F' \
                             WHEN 'SD' THEN 'F' \
                             WHEN 'R' THEN 'R' \
                                 END AS equipment, \
                         CASE accessorial1 WHEN 'FUEL' THEN CAST(carrier_accessorial_rate1 as double precision) END AS one, \
                         CASE accessorial2 WHEN 'FUEL' THEN CAST(carrier_accessorial_rate2 as double precision) END AS two, \
                         CASE accessorial3 WHEN 'FUEL' THEN CAST(carrier_accessorial_rate3 as double precision) END AS three,\
                         CASE accessorial4 WHEN 'FUEL' THEN CAST(carrier_accessorial_rate4 as double precision) END AS four, \
                         CASE accessorial5 WHEN 'FUEL' THEN CAST(carrier_accessorial_rate5 as double precision) END AS five, \
                         CASE accessorial6 WHEN 'FUEL' THEN CAST(carrier_accessorial_rate6 as double precision) END AS six, \
                         CASE accessorial7 WHEN 'FUEL' THEN CAST(carrier_accessorial_rate7 as double precision) END AS seven, \
                         CASE accessorial8 WHEN 'FUEL' THEN CAST(carrier_accessorial_rate8 as double precision) END AS eight\
                         FROM aljex_load \
                         WHERE pickup_date >='{start_date}' \
                         AND pickup_date <='{end_date}' \
                         AND status IN ('RELEASED', 'DELIVERED')\
                         AND equipment IN ('V','VF','VR','F','FSD','FT','SD','R') \
                         AND office NOT IN ('BV','CC','CL','LN','NO','VA','VE')", conn)

In [ ]:
#fill in null values with 0
historical_data = historical_data.fillna(0)

In [ ]:
#convert miles string to an integer
historical_data['miles'] = historical_data['miles'].astype(str).astype(int)

In [ ]:
#filter out the 0 miles loads
historical_data = historical_data[historical_data['miles']>0]

In [ ]:
#create an all-in rate column
historical_data['All in Rate'] = historical_data['carrier_line_haul'] + historical_data['one'] + historical_data['two'] + historical_data['three'] + historical_data['four'] + historical_data['five'] + historical_data['six'] + historical_data['seven'] + historical_data['eight']

In [ ]:
#drop unnecessary columns
historical_data = historical_data[['id',
                                   'origin_city', 
                                   'origin_state',
                                   'pickup_zip_code',
                                   'dest_city',
                                   'dest_state',
                                   'consignee_zip_code',
                                   'pickup_date',
                                   'miles',
                                   'equipment',
                                   'All in Rate']]

In [ ]:
#rename columns
historical_data = historical_data.rename(columns = {'id':'Pro #',
                                                   'origin_city':'Orig City',
                                                   'origin_state':'Orig State',
                                                   'pickup_zip_code':'Pickup Zip',
                                                   'dest_city':'Dest City',
                                                   'dest_state':'Dest State',
                                                   'consignee_zip_code':'Dest Zip',
                                                   'miles':'Miles',
                                                   'pickup_date':'Ship Date', 
                                                   'equipment':'Truck Type'})

In [ ]:
#create combined city, state for origin and destinations
historical_data['Origin'] = historical_data['Orig City'] + ", " + historical_data['Orig State']
historical_data['Dest'] = historical_data['Dest City'] + ", " + historical_data['Dest State']

In [ ]:
#read in the excel bid template
bid_data = pd.read_excel('radius template.xlsx')

In [ ]:
#concatenate origin and destination city/state
bid_data['Origin'] = bid_data['Orig City'] + ", " + bid_data['Orig State']
bid_data['Dest'] = bid_data['Dest City'] + ", " + bid_data['Dest State']

In [ ]:
#create a comprehensive list of historical data and bid data cities
cities = pd.concat([historical_data['Origin'],historical_data['Dest'], bid_data['Origin'], bid_data['Dest']]).to_frame()

In [ ]:
#rename the column in the newly created dataframe from 0 to City
cities = cities.rename(columns = {0:'City'})

In [ ]:
#read in the coordinates that were previously stored.
stored_coordinates = pd.read_csv('stored_records.csv')

In [ ]:
#merge the cities and stored coordinates dataframes to see what records we have already collected data on
cities = cities.merge(stored_coordinates, left_on='City', right_on='Original City', how='left')

In [ ]:
#drop unnecessary columns from the dataframe
cities = cities.drop(columns = ['Original City','Returned City'])

In [ ]:
#grab all of the cities that don't have saved records
cities = cities.loc[cities['Lat'].isnull()]['City'].to_frame()

In [ ]:
#create a unique list of new cities that we don't have saved records for.
new_cities = list(cities['City'].unique())

In [ ]:
#iterate through the list of new cities grabing the geolocation of each and appending them to the stored_coordinates dataframe
for x in range(0,len(new_cities)):
    try:
        city = new_cities[x]
        results = gmaps.geocode(city)
        address = results[0]['formatted_address']
        lat = results[0]['geometry']['location']['lat']
        lng = results[0]['geometry']['location']['lng']
        stored_coordinates = stored_coordinates.append({'Original City':city, 'Returned City':address,'Lat':lat, 'Lng':lng}, ignore_index=True)
        print(f'{x} of {len(new_cities)}')
    except IndexError:
        print(city)

In [ ]:
#save out the stored coordinates dataframe with the new records so we can access it again next time
stored_coordinates.to_csv('stored_records.csv', index=False)

In [ ]:
#drop the returned city column from the dataframe
stored_coordinates = stored_coordinates.drop(columns = 'Returned City')

In [ ]:
# Define a basic Haversine distance formula to calculate distance in miles between origin and destination lat/lon
#formula came from https://engineering.upside.com/a-beginners-guide-to-optimizing-pandas-code-for-speed-c09ef2c6a4d6
#website is also where vectorization is used later to speed up the code
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    total_miles = MILES * c
    return total_miles

In [ ]:
#grab lat, lng for historical origins
historical_data = historical_data.merge(stored_coordinates, left_on='Origin', right_on='Original City')
historical_data = historical_data.drop(columns = ['Original City'])
historical_data = historical_data.rename(columns = {'Lat':'O Lat', 'Lng':'O Lng'})

In [ ]:
#grab lat, lng for historical destinations
historical_data = historical_data.merge(stored_coordinates, left_on='Dest', right_on='Original City')
historical_data = historical_data.drop(columns = ['Original City'])
historical_data = historical_data.rename(columns = {'Lat':'D Lat', 'Lng':'D Lng'})

In [ ]:
#grab lat, lng for bid origins
bid_data = bid_data.merge(stored_coordinates, left_on='Origin', right_on='Original City')
bid_data = bid_data.drop(columns = ['Original City'])
bid_data = bid_data.rename(columns = {'Lat':'O Lat', 'Lng':'O Lng'})

In [ ]:
#grab lat, lng for bid destinations
bid_data = bid_data.merge(stored_coordinates, left_on='Dest', right_on='Original City')
bid_data = bid_data.drop(columns = ['Original City'])
bid_data = bid_data.rename(columns = {'Lat':'D Lat', 'Lng':'D Lng'})

In [ ]:
#define the first quarter start date
quarter_start_date = start_date

In [ ]:
#define a dictionary to help advance quarters iteratively
quarter_advance = {1:{'new_month':4, 'years':0},
                  4:{'new_month':7, 'years':0},
                  7:{'new_month':10, 'years':0},
                  10:{'new_month':1, 'years':1}}

In [ ]:
#define an empty dictionary used in the following iteration.  Will calculate every date span and apend to this dictionary
timespan_calculations = {}

#6 iterations, one for each of the 4 full quarters, one for the 5th QTD, and one for the full-year timespan
for x in range (0, 6):
    #if we are on the 6th iteration, perform a 1-year calculation instead of a quarterly calculation
    if x == 5:
        
        #grab the end date defined above
        quarter_end_date = end_date
        
        #calcualte 1 year prior for a one year timespan
        quarter_start_date = end_date.replace(year = end_date.year-1) + timedelta(days = 1)
        
        #set the timespan variable to 1 year
        timespan = '1 year'
        
    #if we are on iteration five, grab the end date defined above instead of the quarter end as we 
    #are likely in the middle of the quarter and need a QTD calcualation instead
    elif x == 4:
        
        #grab the end date defined above
        quarter_end_date = end_date
        
        #grab the quarter and the year to append to the columns in the dataframe also adding QTD to the variable
        timespan = f"{quarter_start[quarter_start_date.month]['Quarter']} {quarter_start_date.year} QTD"
    
    #for all other iterations A.K.A. the first four quarters, calculate quarter start and quarter end
    else:
        
        #define the quarter end date
        quarter_end_date = quarter_start_date.replace(month = quarter_advance[quarter_start_date.month]['new_month'], 
            year = quarter_start_date.year + quarter_advance[quarter_start_date.month]['years']) - timedelta(days = 1)

        #grab the quarter and the year to append to the columns in the dataframe
        timespan = f"{quarter_start[quarter_start_date.month]['Quarter']} {quarter_start_date.year}"

    #add the compiled data to the timespan_calculations dictionary
    timespan_calculations[timespan] = {'start_date':quarter_start_date, 'end_date':quarter_end_date}
    
    #override the previously existing quarter start date and advancing it to the start date of the next quarter
    quarter_start_date = quarter_end_date + timedelta(days = 1)

In [ ]:
#grab a list of unique timespans
timespans = list(timespan_calculations.keys())

In [ ]:
#can be deleted, used for testing
lat1 = bid_data.iloc[0]['O Lat']
lng1 = bid_data.iloc[0]['O Lng']
lat2 = bid_data.iloc[0]['D Lat']
lng2 = bid_data.iloc[0]['D Lng']
equipment = bid_data.iloc[0]['Truck Type']

In [ ]:
def quarterly_averages(lane_df):
    df = lane_df[(lane_df['Ship Date'] >= timespan_calculations[timespans[0]]['start_date']) & 
            (lane_df['Ship Date'] <= timespan_calculations[timespans[0]]['end_date'])]
    average0 = df['All in Rate'].mean()
    loads0 = len(df)
    df = lane_df[(lane_df['Ship Date'] >= timespan_calculations[timespans[1]]['start_date']) & 
            (lane_df['Ship Date'] <= timespan_calculations[timespans[1]]['end_date'])]
    average1 = df['All in Rate'].mean()
    loads1 = len(df)
    df = lane_df[(lane_df['Ship Date'] >= timespan_calculations[timespans[2]]['start_date']) & 
            (lane_df['Ship Date'] <= timespan_calculations[timespans[2]]['end_date'])]
    average2 = df['All in Rate'].mean()
    loads2 = len(df)
    df = lane_df[(lane_df['Ship Date'] >= timespan_calculations[timespans[3]]['start_date']) & 
            (lane_df['Ship Date'] <= timespan_calculations[timespans[3]]['end_date'])]
    average3 = df['All in Rate'].mean()
    loads3 = len(df)
    df = lane_df[(lane_df['Ship Date'] >= timespan_calculations[timespans[4]]['start_date']) & 
            (lane_df['Ship Date'] <= timespan_calculations[timespans[4]]['end_date'])]
    average4 = df['All in Rate'].mean()
    loads4 = len(df)
    df = lane_df[(lane_df['Ship Date'] >= timespan_calculations[timespans[5]]['start_date']) & 
            (lane_df['Ship Date'] <= timespan_calculations[timespans[5]]['end_date'])]
    average5 = df['All in Rate'].mean()
    loads5 = len(df)
    return average0, loads0, average1, loads1, average2, loads2, average3, loads3, average4, loads4, average5, loads5

In [ ]:
#define a function to grab all loads within pre-defined radius, taking equipment into account
def radius_pull(lat1, lng1, lat2, lng2, equipment):
    
    #grab only the data that matched equipment type
    origin_radius_df = historical_data.loc[historical_data['Truck Type'] == equipment]
    
    #vectorize the haversine function, calculating how far historical miles are from bid origin, storing results in a new column
    origin_radius_df['O Distance'] = haversine(lat1, lng1, origin_radius_df['O Lat'], origin_radius_df['O Lng'])
    
    #filter the origin radius dataframe to only have Origin distance less than specified distance above, store in new df
    dest_radius_df = origin_radius_df.loc[origin_radius_df['O Distance']<distance]
    
    #of the remaining results, calculate how far historical miles are from bid destination, storing results in a new column
    dest_radius_df['D Distance'] = haversine(lat2, lng2, dest_radius_df['D Lat'], dest_radius_df['D Lng'])
    
    #filter the dest radius dataframe to only have dest distance less than specified distance above, store in new df
    lane_df = dest_radius_df.loc[dest_radius_df['D Distance']<distance]
    
    average0, loads0, average1, loads1, average2, loads2, average3, loads3, average4, loads4, average5, loads5 = quarterly_averages(lane_df)
    
    #spit out variables
    return lane_df, average0, loads0, average1, loads1, average2, loads2, average3, loads3, average4, loads4, average5, loads5

In [ ]:
#create empty columns in the bid_data df
bid_data[f'{timespans[0]} Avg. Rate'] = ""
bid_data[f'{timespans[0]} Loads'] = ""
bid_data[f'{timespans[1]} Avg. Rate'] = ""
bid_data[f'{timespans[1]} Loads'] = ""
bid_data[f'{timespans[2]} Avg. Rate'] = ""
bid_data[f'{timespans[2]} Loads'] = ""
bid_data[f'{timespans[3]} Avg. Rate'] = ""
bid_data[f'{timespans[3]} Loads'] = ""
bid_data[f'{timespans[4]} Avg. Rate'] = ""
bid_data[f'{timespans[4]} Loads'] = ""
bid_data[f'{timespans[5]} Avg. Rate'] = ""
bid_data[f'{timespans[5]} Loads'] = ""
bid_data['Lane'] = bid_data['Origin'] + ' to ' + bid_data['Dest']

In [ ]:
#create an empty dataframe
final_df = pd.DataFrame()
#iterate through the rows int he bid_data df performing the radius_pull function for each row
for index, row in bid_data.iterrows():
    #store the variables spit out of the radius_pull function
    lane_df, average0, loads0, average1, loads1, average2, loads2, average3, loads3, average4, loads4, average5, loads5 = radius_pull(row['O Lat'], row['O Lng'], row['D Lat'], row['D Lng'], row['Truck Type'])
    #using the .loc function with the index, set the average rate to the variable from the radius_pull function
    bid_data.loc[index, f'{timespans[0]} Avg. Rate'] = average0
    bid_data.loc[index, f'{timespans[1]} Avg. Rate'] = average1
    bid_data.loc[index, f'{timespans[2]} Avg. Rate'] = average2
    bid_data.loc[index, f'{timespans[3]} Avg. Rate'] = average3
    bid_data.loc[index, f'{timespans[4]} Avg. Rate'] = average4
    bid_data.loc[index, f'{timespans[5]} Avg. Rate'] = average5
    
    #using the .loc function with the index, set the load count to the variable from the radius_pull function
    bid_data.loc[index, f'{timespans[0]} Loads'] = loads0
    bid_data.loc[index, f'{timespans[1]} Loads'] = loads1
    bid_data.loc[index, f'{timespans[2]} Loads'] = loads2
    bid_data.loc[index, f'{timespans[3]} Loads'] = loads3
    bid_data.loc[index, f'{timespans[4]} Loads'] = loads4
    bid_data.loc[index, f'{timespans[5]} Loads'] = loads5
    
    #in the lane_df, set the bid lane to the current lane in the iterrows function
    lane_df['Bid Lane'] = row['Lane']
    
    #append the lane_df to the final_df
    final_df = final_df.append([lane_df])
    print(index)

In [ ]:
#save the new bid_data df
bid_data.to_csv('historicals.csv', index=False)

In [ ]:
#save the new final_data df
final_df.to_csv('lane_data.csv', index=False)

In [ ]:
#grab an ending 
end = datetime.now()

In [ ]:
#calculate script runtime
end-start